In [3]:
from pathlib import Path

from ssd import SSD

In [4]:
VAL_IMAGES_DIR = Path("/mnt/data/datasets/object_detection/coco/images/val2017")
VAL_LABELS_DIR = Path("/mnt/data/datasets/object_detection/coco/labels/val2017")

In [11]:
# Find all validation images and labels - make sure both exist
val_image_paths = list(VAL_IMAGES_DIR.glob("*.jpg"))
val_label_paths = list(VAL_LABELS_DIR.glob("*.txt"))

stem_to_image_path = {f.stem: f for f in val_image_paths}
stem_to_label_path = {f.stem: f for f in val_label_paths}

image_label_pairs: list[tuple[Path, Path]] = []
for stem in stem_to_image_path:
    if stem in stem_to_label_path:
        image_label_pairs.append((stem_to_image_path[stem], stem_to_label_path[stem]))

In [2]:
model = SSD(80)